In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_10014/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

import os

os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

spark = SparkSession.builder \
    .appName("USPS") \
    .config("spark.executor.memory", "8G") \
    .config("spark.driver.memory", "8G") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "4G") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/21 16:46:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/21 16:46:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/21 16:46:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/06/21 16:46:33 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/06/21 16:46:33 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/06/21 16:46:33 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [4]:
df_zip_latlon = spark.read.parquet("/home/pk/DAEN690/zip_latlon.parquet")

In [5]:
df_zip_latlon.show(3)

+-------+---------+----------+
|zipcode|      LAT|       LNG|
+-------+---------+----------+
|  00601|18.180555|-66.749961|
|  00602|18.361945|-67.175597|
|  00603|18.455183|-67.119887|
+-------+---------+----------+
only showing top 3 rows



In [6]:
df_PKG_O = spark.read.parquet("/home/pk/DAEN690/PKG_Origin_cleaned_YM.parquet")

In [7]:
df_PKG_O.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+
|85660637209|           020 |     42240|           2024-01-22|          98230|           G0|         16|                 5|                  12|            169920| 2023-09-26|      2024-01-22|   0|       202309|       202401|
|85684177804|           020 |     42141|           2024-01-12|          91715|           G0|    

In [8]:
from pyspark.sql.functions import min
min_date_Ori = df_PKG_O.agg(min("Origin_date")).first()[0]
min_date_Des = df_PKG_O.agg(min("destination_date")).first()[0]

# Show the minimum date
print("Minimum Date Ori:", min_date_Ori," --- ","Minimum Date Des:", min_date_Des )
print(df_PKG_O.count())

Minimum Date Ori: 1969-12-31  ---  Minimum Date Des: 2024-01-08
1115279


In [10]:
df_PKG_O = df_PKG_O.join(df_zip_latlon, df_PKG_O["Origin_zip"] == df_zip_latlon["zipcode"], "left")

In [11]:
df_PKG_O.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+-------+---------+----------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|zipcode|      LAT|       LNG|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+-------+---------+----------+
|85660637209|           020 |     42240|           2024-01-22|          98230|           G0|         16|                 5|                  12|            169920| 2023-09-26|      2024-01-22|   0|       202309|       202401|  42240|36

In [12]:
df_PKG_O.count()

1115279

In [13]:
df_PKG_O = df_PKG_O.drop("zipcode")
dict_rename = {"LAT": "LAT_O", "LNG": "LNG_O"}

for old_name, new_name in dict_rename.items():
    df_PKG_O = df_PKG_O.withColumnRenamed(old_name, new_name)

df_PKG_O.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+
|85660637209|           020 |     42240|           2024-01-22|          98230|           G0|         16|                 5|                  12|            169920| 2023-09-26|      2024-01-22|   0|       202309|       202401|36.880182|-87.463142|
|85684177804

In [14]:
df_PKG_O = df_PKG_O.join(df_zip_latlon, df_PKG_O["destination_zip"] == df_zip_latlon["zipcode"], "left")

In [15]:
df_PKG_O.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+-------+---------+-----------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|zipcode|      LAT|        LNG|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+-------+---------+-----------+
|85660637209|           020 |     42240|           2024-01-22|          98230|           G0|         16|                 5|                  12|            169920| 2023-

In [16]:
df_PKG_O = df_PKG_O.drop("zipcode")
dict_rename = {"LAT": "LAT_D", "LNG": "LNG_D"}

for old_name, new_name in dict_rename.items():
    df_PKG_O = df_PKG_O.withColumnRenamed(old_name, new_name)

df_PKG_O.show(3)
df_PKG_O.count()

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|    LAT_D|      LNG_D|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+
|85660637209|           020 |     42240|           2024-01-22|          98230|           G0|         16|                 5|                  12|            169920| 2023-09-26|      2024-01-22| 

1115279

In [17]:
########################################################################################################

In [18]:
df_weather = spark.read.csv("combined_storm_events_with_types.csv", header=True)
df_weather.show(3)

+---------+----------+--------+--------------+-----+-------+------------+--------+---------+-------+--------+------------+
|YEARMONTH|EPISODE_ID|EVENT_ID|LOCATION_INDEX|RANGE|AZIMUTH|    LOCATION|LATITUDE|LONGITUDE|   LAT2|    LON2|  EVENT_TYPE|
+---------+----------+--------+--------------+-----+-------+------------+--------+---------+-------+--------+------------+
|   202210|    173111| 1051457|             1|  0.0|      N|    EDGEWOOD|   35.07|   -106.2| 354200|10612000|Funnel Cloud|
|   202210|    173111| 1053281|             1| 9.95|     NW|MEXICAN SPGS| 35.8753|-108.9632|3552518|10857792|Funnel Cloud|
|   202210|    173122| 1051505|             1| 1.08|      W|       NUCLA|   38.27|  -108.55|3816200|10833000|        Hail|
+---------+----------+--------+--------------+-----+-------+------------+--------+---------+-------+--------+------------+
only showing top 3 rows



In [19]:
df1 = df_PKG_O
df2 = df_weather

In [20]:
from pyspark.sql.functions import col, round

df1 = df1.withColumn("LAT_O_ROUND", round(col("LAT_O"), 1)) \
         .withColumn("LNG_O_ROUND", round(col("LNG_O"), 1)) \
         .withColumn("LAT_D_ROUND", round(col("LAT_D"), 1)) \
         .withColumn("LNG_D_ROUND", round(col("LNG_D"), 1)) \

df2 = df2.withColumn("LATITUDE_ROUND", round(col("LATITUDE"), 1)) \
         .withColumn("LONGITUDE_ROUND", round(col("LONGITUDE"), 1))

df1.show(3)
df2.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+-----------+-----------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|    LAT_D|      LNG_D|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+-----------+-----------+
|85660637209|           020 |     42240|         

In [21]:
result_df = df1.join(
    df2,
    (df1["LAT_O_ROUND"] == df2["LATITUDE_ROUND"]) &
    (df1["LNG_O_ROUND"] == df2["LONGITUDE_ROUND"]) &
    (df1["YEARMONTH_Ori"] == df2["YEARMONTH"]),
    how="left"
).select(
    df1["*"], df2["EVENT_ID"], df2["YEARMONTH"],df2["EVENT_TYPE"]
)

result_df.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+-----------+-----------+--------+---------+-----------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|    LAT_D|      LNG_D|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|EVENT_ID|YEARMONTH| EVENT_TYPE|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+-----------+

In [22]:
result_df.count()
# 1115279 original size
# new size 1115489

1115489

In [25]:
from pyspark.sql.functions import col, count


duplicate_ids = (result_df.groupBy("MailPieceID")
                  .agg(count("MailPieceID").alias("count"))
                  .filter(col("count") > 1)
                  .select("MailPieceID"))

duplicate_rows = result_df.join(duplicate_ids, on="MailPieceID", how="inner")

result_val = duplicate_rows.select("MailPieceID", "EVENT_ID")

result_val.show()

+-----------+--------+
|MailPieceID|EVENT_ID|
+-----------+--------+
|85660637209| 1133997|
|85660637209| 1133996|
|85660637209| 1133996|
|85660637209| 1130760|
|87652110395| 1144555|
|87652110395| 1144554|
|87896824000| 1144555|
|87896824000| 1144554|
|87924089178| 1144555|
|87924089178| 1144554|
|87935789929| 1144555|
|87935789929| 1144554|
|87935789930| 1144555|
|87935789930| 1144554|
|87935789935| 1144555|
|87935789935| 1144554|
|87935789948| 1144555|
|87935789948| 1144554|
|87935789949| 1144555|
|87935789949| 1144554|
+-----------+--------+
only showing top 20 rows



In [25]:
dict_rename = {"EVENT_ID": "EVENT_ID_O","YEARMONTH":"YEARMONTH_O", "EVENT_TYPE": "EVENT_TYPE_O"}

for old_name, new_name in dict_rename.items():
    result_df = result_df.withColumnRenamed(old_name, new_name)

result_df.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+-----------+-----------+----------+-----------+------------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|    LAT_D|      LNG_D|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|EVENT_ID_O|YEARMONTH_O|EVENT_TYPE_O|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+--

In [26]:
df2.show(3)

+---------+----------+--------+--------------+-----+-------+------------+--------+---------+-------+--------+------------+--------------+---------------+
|YEARMONTH|EPISODE_ID|EVENT_ID|LOCATION_INDEX|RANGE|AZIMUTH|    LOCATION|LATITUDE|LONGITUDE|   LAT2|    LON2|  EVENT_TYPE|LATITUDE_ROUND|LONGITUDE_ROUND|
+---------+----------+--------+--------------+-----+-------+------------+--------+---------+-------+--------+------------+--------------+---------------+
|   202210|    173111| 1051457|             1|  0.0|      N|    EDGEWOOD|   35.07|   -106.2| 354200|10612000|Funnel Cloud|          35.1|         -106.2|
|   202210|    173111| 1053281|             1| 9.95|     NW|MEXICAN SPGS| 35.8753|-108.9632|3552518|10857792|Funnel Cloud|          35.9|         -109.0|
|   202210|    173122| 1051505|             1| 1.08|      W|       NUCLA|   38.27|  -108.55|3816200|10833000|        Hail|          38.3|         -108.6|
+---------+----------+--------+--------------+-----+-------+------------+---

In [27]:
from pyspark.sql import functions as F

result_df_alias = result_df.alias("result_df")
df2_alias = df2.alias("df2")

result_df2 = result_df_alias.join(
    df2_alias,
    (result_df_alias["LAT_D_ROUND"] == df2_alias["LATITUDE_ROUND"]) &
    (result_df_alias["LNG_D_ROUND"] == df2_alias["LONGITUDE_ROUND"]) &
    (result_df_alias["YEARMONTH_Des"] == df2_alias["YEARMONTH"]),
    how="left"
).select(
    [F.col("result_df.*")] + [F.col("df2.EVENT_ID"), F.col("df2.YEARMONTH"), F.col("df2.EVENT_TYPE")]
)

result_df2.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+-----------+-----------+----------+-----------+------------+--------+---------+----------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|    LAT_D|      LNG_D|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|EVENT_ID_O|YEARMONTH_O|EVENT_TYPE_O|EVENT_ID|YEARMONTH|EVENT_TYPE|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------

In [29]:
result_df2.count()
# 1115279 original size
# new size 1115489, 1228246

1228246

In [26]:
dict_rename = {"EVENT_ID": "EVENT_ID_D","YEARMONTH":"YEARMONTH_D", "EVENT_TYPE": "EVENT_TYPE_D"}

for old_name, new_name in dict_rename.items():
    result_df2 = result_df2.withColumnRenamed(old_name, new_name)

result_df2.show(3)

+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-------------+---------+----------+---------+-----------+-----------+-----------+-----------+-----------+----------+-----------+------------+----------+-----------+------------+
|MailPieceID|ServiceTypeCode|Origin_zip|ScheduledDeliveryDate|destination_zip|MailClassCode|total_scans|Distinct_zip_scans|Distinct_event_scans|time_delta_minutes|Origin_date|destination_date|late|YEARMONTH_Ori|YEARMONTH_Des|    LAT_O|     LNG_O|    LAT_D|      LNG_D|LAT_O_ROUND|LNG_O_ROUND|LAT_D_ROUND|LNG_D_ROUND|EVENT_ID_O|YEARMONTH_O|EVENT_TYPE_O|EVENT_ID_D|YEARMONTH_D|EVENT_TYPE_D|
+-----------+---------------+----------+---------------------+---------------+-------------+-----------+------------------+--------------------+------------------+-----------+----------------+----+-------------+-----------

In [27]:
result_df2.write.mode('overwrite').parquet("/home/pk/DAEN690/PKG_final.parquet")
print("done")

done


In [28]:
result_df2.count()

1228246

In [29]:
spark.stop()